In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io 
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from scipy.fft import fft
import random
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [2]:

from scipy.io import loadmat
faultI10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_10hz.mat')
faultI30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_30hz.mat')
faultII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_10hz.mat')
faultII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_30hz.mat')
faultIII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_10hz.mat')
faultIII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_30hz.mat')
faultIV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_10hz.mat')
faultIV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_30hz.mat')
faultV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_10hz.mat')
faultV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_30hz.mat')



In [3]:
sigII30=faultII30['Acquisition_un_2_30hz']
sigII10=faultII10['Acquisition_un_2_10hz']
del faultII30,faultII10
sigI30=faultI30['Acquisition_un_1_30hz']
sigI10=faultI10['Acquisition_un_1_10hz']
del faultI30,faultI10
sigIII10=faultIII10['Acquisition_un_3_10hz']
sigIII30=faultIII30['Acquisition_un_3_30hz']
del faultIII10,faultIII30
sigIV10=faultIV10['Acquisition_un_4_10hz']
sigIV30=faultIV30['Acquisition_un_4_30hz']
del faultIV10,faultIV30
sigV10=faultV10['Acquisition_un_5_10hz']
sigV30=faultV30['Acquisition_un_5_30hz']
del faultV10,faultV30
# sigVI10=faultVI10['Acquisition_un_6_10hz']
# sigVI30=faultVI30['Acquisition_un_6_30hz']
#  del faultVI10,faultVI30

In [4]:

def sliding_window(data, window_size, stride, training_size):
    """
    Applies a sliding window to the given data with the specified window size
    and stride length. Padding is added at the beginning and end of the data to
    ensure all samples are included in windows.
    """
    num_padding = window_size - stride
    data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    window_data = []
    for i in range(0, len(data_padded) - window_size + 1, stride):
        window = data_padded[i:i+window_size, :]
        window_data.append(window)
    window_data = np.array(window_data)
    window_data = window_data[:training_size, :, :]
    window_data = np.transpose(window_data, (0, 2, 1))
    return window_data

In [5]:
windowsize=1500
stride=1400

In [6]:
sigtI10hz=sliding_window(sigI10,windowsize,stride,40000)[:,1:13,:]
sigtI30hz=sliding_window(sigI30,windowsize,stride,40000)[:,1:13,:]
sigtII10hz=sliding_window(sigII10,windowsize,stride,40000)[:,1:13,:]
sigtII30hz=sliding_window(sigII30,windowsize,stride,40000)[:,1:13,:]
del sigI10,sigI30,sigII10,sigII30
sigtIII30hz=sliding_window(sigIII30,windowsize,stride,40000)[:,1:13,:]
sigtIII10hz=sliding_window(sigIII10,windowsize,stride,40000)[:,1:13,:]
sigtIV30hz=sliding_window(sigIV30,windowsize,stride,40000)[:,1:13,:]
sigtIV10hz=sliding_window(sigIV10,windowsize,stride,40000)[:,1:13,:]
del sigIII30,sigIII10,sigIV30,sigIV10
sigtV30hz=sliding_window(sigV30,windowsize,stride,40000)[:,1:13,:]
sigtV10hz=sliding_window(sigV10,windowsize,stride,40000)[:,1:13,:]
# sigtVI30hz=sliding_window(sigVI30,windowsize,stride,40000)
# sigtVI10hz=sliding_window(sigVI10,windowsize,stride,40000)
# del sigV30,sigV10,sigVI30,sigVI10

In [7]:
datalable1=np.concatenate([sigtI10hz,sigtI30hz],axis=-1)

datalable2=np.concatenate([sigtII10hz,sigtII30hz],axis=-1)

datalable3=np.concatenate([sigtIII10hz,sigtIII30hz],axis=-1)

datalable4=np.concatenate([sigtIV10hz,sigtIV30hz],axis=-1)

datalabele5=np.concatenate([sigtV10hz,sigtV30hz],axis=-1)

# datalabele6=np.concatenate([sigtBI10,sigtBI30],axis=-1)

# datalabele7=np.concatenate([sigtBII10,sigtBII30],axis=-1)

In [8]:
labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)
dataall=np.concatenate([datalable1,datalable2,datalable3,datalable4,datalabele5],axis=0)

In [9]:
np.shape(labels)

(2150,)

In [10]:
indices = np.random.permutation(dataall.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = dataall[indices]
shuffled_labels = labels[indices]
dataall= shuffled_data
labels=shuffled_labels

In [11]:
np.shape(dataall)

(2150, 12, 3000)

In [12]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import DepthwiseConv2D, Flatten, Dense, Dropout

# reshape the data to have 4 dimensions

data = dataall.reshape(dataall.shape[0], dataall.shape[1], dataall.shape[2], 1)

# create labels


# split the data into training and testing sets




2023-05-03 20:37:42.744988: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 20:37:42.893279: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 20:37:42.894188: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 20:37:43.900004: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
np.shape(data)

(2150, 12, 3000, 1)

In [14]:
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)

#create the model
# model = Sequential()
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 3000, 1)))
# model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 3000, 1)))
# model.add(Flatten())
# model.add(Dense(5, activation='softmax'))
model = Sequential()
model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu', input_shape=(12, 3000, 1)))
model.add(Dropout(0.2))
model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu'))
model.add(Dropout(0.2))
model.add(DepthwiseConv2D(kernel_size=(3, 3), strides=(1, 1), depth_multiplier=1, padding='same', activation='relu'))
model.add(Flatten())
model.add(Dense(5, activation='softmax'))


# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(train_data, train_labels, epochs=50, batch_size=32, validation_data=(test_data, test_labels))


2023-05-03 20:37:45.263791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 20:37:45.264618: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/50


2023-05-03 20:37:45.624793: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 247680000 exceeds 10% of free system memory.


54/54 [==============================] - 9s 161ms/step - loss: 1.6101 - accuracy: 0.2070 - val_loss: 1.6096 - val_accuracy: 0.1814
Epoch 2/50
54/54 [==============================] - 8s 148ms/step - loss: 1.5771 - accuracy: 0.3570 - val_loss: 1.6100 - val_accuracy: 0.2023
Epoch 3/50
54/54 [==============================] - 8s 146ms/step - loss: 1.5283 - accuracy: 0.3703 - val_loss: 1.6027 - val_accuracy: 0.4116
Epoch 4/50
54/54 [==============================] - 8s 146ms/step - loss: 1.3917 - accuracy: 0.4064 - val_loss: 1.5158 - val_accuracy: 0.2047
Epoch 5/50
54/54 [==============================] - 8s 148ms/step - loss: 0.9634 - accuracy: 0.6570 - val_loss: 1.2099 - val_accuracy: 0.6535
Epoch 6/50
54/54 [==============================] - 8s 147ms/step - loss: 0.5050 - accuracy: 0.8116 - val_loss: 0.9691 - val_accuracy: 0.6488
Epoch 7/50
54/54 [==============================] - 8s 148ms/step - loss: 0.3976 - accuracy: 0.8227 - val_loss: 0.7732 - val_accuracy: 0.6605
Epoch 8/50
54/54 